#Установка CityGeoTools

In [1]:
!git clone https://github.com/iduprojects/CityGeoTools

Cloning into 'CityGeoTools'...
remote: Enumerating objects: 2775, done.
remote: Counting objects: 100% (1030/1030), done.
remote: Compressing objects: 100% (390/390), done.
remote: Total 2775 (delta 689), reused 850 (delta 636), pack-reused 1745
Receiving objects: 100% (2775/2775), 114.30 MiB | 9.84 MiB/s, done.
Resolving deltas: 100% (1915/1915), done.


In [2]:
!pip install -r /content/CityGeoTools/metrics/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.0/243.0 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install scipy

In [4]:
!pip install rpyc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.0/127.0 kB 5.9 MB/s eta 0:00:00


In [5]:
!pip install networkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 21.8 MB/s eta 0:00:00


In [6]:
import sys
import os
import numpy as np
import geopandas as gpd
import folium
from CityGeoTools.metrics.data import CityInformationModel as BaseModel

#Модель города

In [7]:
city_model = BaseModel.CityInformationModel(city_name="Ekaterinburg", city_crs=32641, cwd="CityGeoTools")

#Здания

In [ ]:
buildings = gpd.read_file('/content/New_building_input.geojson')
blocks = gpd.read_file('/content/city_blocks.geojson')

In [9]:
buildings = buildings.to_crs(epsg=32641)
blocks = blocks.to_crs(buildings.crs)

In [10]:
buildings['basement_area'] = buildings.geometry.area

In [11]:
buildings["id"] = range(len(buildings))
blocks["block_id"] = range(len(blocks))

In [12]:
buildings = buildings.sjoin(blocks, how="left")
buildings = buildings.drop(columns=["index_right"])

In [13]:
buildings["block_id"] = buildings["block_id"].fillna(buildings["block_id"].median())

In [14]:
buildings = buildings.astype({"is_living": bool, "basement_area": int, "block_id": int})

In [15]:
buildings = buildings.rename(columns={"id_left": "id", "area_residential": "living_area","building:levels": "storeys_count"})

In [16]:
buildings = buildings.to_crs(epsg=4326)
buildings.crs = 'urn:ogc:def:crs:OGC::CRS84'

columns_to_keep = ["id", "is_living", "basement_area", "living_area", "storeys_count", "block_id", "geometry"]
buildings = buildings[columns_to_keep]

buildings.to_file('Building.geojson', driver='GeoJSON')

In [17]:
city_model.update_layer(attr_name="Buildings", file_name="/content/Building.geojson")

print(city_model.Buildings)

Validation of Buildings layer...
Buildings layer loaded successfully!
                                                geometry     id  is_living  \
0      POLYGON ((356049.935 6303683.651, 356043.965 6...      0      False   
1      POLYGON ((344479.927 6284407.547, 344484.218 6...      1      False   
2      POLYGON ((344521.671 6284424.151, 344527.278 6...      2      False   
3      POLYGON ((344640.423 6284405.961, 344646.449 6...      3      False   
4      POLYGON ((355911.449 6304877.799, 355911.015 6...      4      False   
...                                                  ...    ...        ...   
47942  POLYGON ((351568.269 6283118.841, 351573.230 6...  47937      False   
47943  POLYGON ((351747.536 6282871.276, 351753.004 6...  47938      False   
47944  POLYGON ((350661.800 6295344.896, 350670.493 6...  47939      False   
47945  POLYGON ((356355.355 6297600.006, 356353.781 6...  47940      False   
47946  POLYGON ((356339.918 6297770.965, 356332.771 6...  47941      Fal

In [18]:
city_model.methods.spacematrix.message

{'Buildings': 'Layer matches specification'}

#Кварталы

In [ ]:
blocks = gpd.read_file('/content/city_blocks.geojson')
districts = gpd.read_file('/content/districts.geojson')

In [20]:
districts = districts.to_crs(epsg=32641)
blocks = blocks.to_crs(districts.crs)

In [21]:
blocks['basement_area'] = blocks.geometry.area
blocks['municipality_id'] = 10

In [22]:
blocks["block_id"] = range(len(blocks))
districts["district_id"] = range(len(districts))

In [23]:
blocks = blocks.sjoin(districts, how="left")
blocks = blocks.drop_duplicates(subset='block_id')
blocks["district_id"] = blocks["district_id"].fillna(blocks["district_id"].median())

In [24]:
blocks

,id_left,geometry,basement_area,municipality_id,block_id,index_right,id_right,name,population_2021,district_id
0,0,"POLYGON ((337102.644 6282010.350, 337066.545 6...",2.088439e+04,10,0,2.0,relation/5817295,Чкаловский район,286277.0,2.0
1,0,"POLYGON ((332586.397 6285543.811, 332586.295 6...",5.350815e+03,10,1,2.0,relation/5817295,Чкаловский район,286277.0,2.0
2,7,"POLYGON ((327934.881 6295918.459, 327935.051 6...",8.435762e+07,10,2,2.0,relation/5817295,Чкаловский район,286277.0,2.0
3,11,"POLYGON ((327896.327 6291395.938, 327896.341 6...",2.151798e+04,10,3,0.0,relation/5803327,Верх-Исетский район,240822.0,0.0
4,12,"POLYGON ((327788.606 6291898.023, 327788.748 6...",1.385571e+04,10,4,0.0,relation/5803327,Верх-Исетский район,240822.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2007,2454,"POLYGON ((371485.042 6295848.882, 371484.976 6...",1.012257e+04,10,2007,1.0,relation/5803648,Октябрьский район,151775.0,1.0
2008,2455,"POLYGON ((371506.950 6295874.962, 371506.876 6...",1.981268e+04,10,2008,1.0,relation/5803648,Октябрьский район,151775.0,1.0
2009,2456,"POLYGON ((371421.206 6296084.046, 371421.144 6...",2.657754e+04,10,2009,1.0,relation/5803648,Октябрьский район,151775.0,1.0
2010,2457,"POLYGON ((370516.260 6296097.439, 370552.821 6...",3.533232e+05,10,2010,1.0,relation/5803648,Октябрьский район,151775.0,1.0


In [25]:
blocks = blocks.drop(columns=["id_left","index_right","population_2021","population_2021", "id_right", "name"])

In [26]:
blocks

,geometry,basement_area,municipality_id,block_id,district_id
0,"POLYGON ((337102.644 6282010.350, 337066.545 6...",2.088439e+04,10,0,2.0
1,"POLYGON ((332586.397 6285543.811, 332586.295 6...",5.350815e+03,10,1,2.0
2,"POLYGON ((327934.881 6295918.459, 327935.051 6...",8.435762e+07,10,2,2.0
3,"POLYGON ((327896.327 6291395.938, 327896.341 6...",2.151798e+04,10,3,0.0
4,"POLYGON ((327788.606 6291898.023, 327788.748 6...",1.385571e+04,10,4,0.0
...,...,...,...,...,...
2007,"POLYGON ((371485.042 6295848.882, 371484.976 6...",1.012257e+04,10,2007,1.0
2008,"POLYGON ((371506.950 6295874.962, 371506.876 6...",1.981268e+04,10,2008,1.0
2009,"POLYGON ((371421.206 6296084.046, 371421.144 6...",2.657754e+04,10,2009,1.0
2010,"POLYGON ((370516.260 6296097.439, 370552.821 6...",3.533232e+05,10,2010,1.0


In [27]:
blocks = blocks.rename(columns={"block_id":"id", "district_id":"administrative_unit_id", "basement_area":"area"})

In [28]:
blocks['administrative_unit_id'] = blocks['administrative_unit_id'].astype(int)

In [29]:
blocks

,geometry,area,municipality_id,id,administrative_unit_id
0,"POLYGON ((337102.644 6282010.350, 337066.545 6...",2.088439e+04,10,0,2
1,"POLYGON ((332586.397 6285543.811, 332586.295 6...",5.350815e+03,10,1,2
2,"POLYGON ((327934.881 6295918.459, 327935.051 6...",8.435762e+07,10,2,2
3,"POLYGON ((327896.327 6291395.938, 327896.341 6...",2.151798e+04,10,3,0
4,"POLYGON ((327788.606 6291898.023, 327788.748 6...",1.385571e+04,10,4,0
...,...,...,...,...,...
2007,"POLYGON ((371485.042 6295848.882, 371484.976 6...",1.012257e+04,10,2007,1
2008,"POLYGON ((371506.950 6295874.962, 371506.876 6...",1.981268e+04,10,2008,1
2009,"POLYGON ((371421.206 6296084.046, 371421.144 6...",2.657754e+04,10,2009,1
2010,"POLYGON ((370516.260 6296097.439, 370552.821 6...",3.533232e+05,10,2010,1


In [30]:
blocks = blocks.to_crs(epsg=4326)
blocks.crs = 'urn:ogc:def:crs:OGC::CRS84'

blocks.to_file("blocks.geojson", driver="GeoJSON")

In [31]:
city_model.update_layer(attr_name="Blocks", file_name="/content/blocks.geojson")

print(city_model.Blocks)

Validation of Blocks layer...
Blocks layer loaded successfully!
                                               geometry          area  \
0     POLYGON ((337102.644 6282010.350, 337066.545 6...  2.088439e+04   
1     POLYGON ((332586.397 6285543.811, 332586.295 6...  5.350815e+03   
2     POLYGON ((327934.881 6295918.459, 327935.051 6...  8.435762e+07   
3     POLYGON ((327896.327 6291395.938, 327896.341 6...  2.151798e+04   
4     POLYGON ((327788.606 6291898.023, 327788.748 6...  1.385571e+04   
...                                                 ...           ...   
2007  POLYGON ((371485.042 6295848.882, 371484.976 6...  1.012257e+04   
2008  POLYGON ((371506.950 6295874.962, 371506.876 6...  1.981268e+04   
2009  POLYGON ((371421.206 6296084.046, 371421.144 6...  2.657754e+04   
2010  POLYGON ((370516.260 6296097.439, 370552.821 6...  3.533232e+05   
2011  POLYGON ((365114.758 6298108.845, 365114.753 6...  5.239537e+07   

      municipality_id    id  administrative_unit_id  
0    

In [32]:
city_model.methods.spacematrix.message

{'Buildings': 'Layer matches specification',
 'Blocks': 'Layer matches specification'}